In [ ]:
import copy
from typing import Union, List, Dict, Any

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.metrics import make_scorer, mean_absolute_percentage_error

from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor


plt.rcParams['axes.edgecolor'] = 'blue'
plt.rcParams['figure.figsize'] = [14, 7]

In [2]:
df_rrs = pd.read_csv("../data/processed/rrs_data.csv", index_col=0)
df_rrs = df_rrs[df_rrs['CHL'] < df_rrs['CHL'].quantile(0.97)]

df_rrs_suppl = copy.copy(df_rrs)
df_rrs_suppl['Ratio B5/B4'] = df_rrs['B5'] / df_rrs['B4']
df_rrs_suppl['NIRRI'] = df_rrs['B8A'] / df_rrs['B4']
df_rrs_suppl['NDCI'] = (1/df_rrs['B5'] - 1/df_rrs['B4']) / (1/df_rrs['B5'] + 1/df_rrs['B4'])
df_rrs_suppl['BGI'] = df_rrs['B2'] / df_rrs['B3']
df_rrs_suppl['GNDVI'] = (df_rrs['B7'] - df_rrs['B3']) / (df_rrs['B7'] + df_rrs['B3'])
df_rrs_suppl['NDTI'] = (df_rrs['B4'] - df_rrs['B3']) / (df_rrs['B4'] + df_rrs['B3'])
df_rrs_suppl['NRDCI'] = (df_rrs['B5'] - df_rrs['B4']) / (df_rrs['B5'] + df_rrs['B4'])
df_rrs_suppl['Ratio B3/B8'] = df_rrs['B3'] / df_rrs['B8']
df_rrs_suppl['Ratio B3/B8A'] = df_rrs['B3'] / df_rrs['B8A']

In [27]:
cols = df_rrs_suppl.columns.tolist()
cols.remove('CHL')
cols.append('CHL')
df_rrs_suppl = df_rrs_suppl[cols]

## Определение используемых функций

In [ ]:
def bias(in_situ: Union[List[float], np.ndarray], pred: Union[List[float], np.ndarray]) -> float:
    """
    Вычисляет систематическую ошибку (Bias) между истинными и предсказанными значениями.

    Args:
    -----------
    in_situ : Union[List[float], np.ndarray]
        Истинные значения целевой переменной
    pred : Union[List[float], np.ndarray]
        Предсказанные значения целевой переменной

    Returns:
    --------
    float
        Среднее значение разности между предсказаниями и истинными значениями
    """
    res = 0
    for i, ai in enumerate(in_situ):
        res += (pred[i] - ai)
    return res / len(in_situ)


def do_cross_validation(estimator, X: Union[pd.DataFrame, np.ndarray], 
                      y: Union[pd.Series, np.ndarray]) -> Dict[str, np.ndarray]:
    """
    Выполняет кросс-валидацию с несколькими метриками оценки.

    Args:
        estimator : BaseEstimator
            Обучаемая модель
        X : Union[pd.DataFrame, np.ndarray]
            Матрица признаков
        y : Union[pd.Series, np.ndarray]
            Целевая переменная

    Returns:
        Dict[str, np.ndarray]
            Словарь с результатами кросс-валидации для каждой метрики
    """
    
    scoring = {
        "Bias": make_scorer(bias, greater_is_better=False),
        "RMSE": "neg_root_mean_squared_error",
        "MAPE": 'neg_mean_absolute_percentage_error'
    }
    
    cv_results = cross_validate(
        estimator, 
        X, 
        y, 
        scoring=scoring, 
        cv=5,
        return_estimator=True
    )
    
    for metric in scoring:
        values = cv_results[f"test_{metric}"]
        print(f"Cross-validated {metric}: ({' + '.join(f'{v:.2f}' for v in values)}) / 5 = {np.mean(values):.2f}")


def nested_cv(df_rrs: pd.DataFrame, name_bands: List[str], param_grid: Dict[str, Any], 
             scoring: Union[str, Dict], model) -> Dict[str, np.ndarray]:
    """
    Выполняет вложенную кросс-валидацию.

    Args:
        df_rrs : pd.DataFrame
            DataFrame с данными (последний столбец содержит целевую переменную)
        name_bands : List[str]
            Список названий используемых спектральных диапазонов
        param_grid : Dict[str, Any]
            Сетка гиперпараметров для поиска
        scoring : Union[str, Dict]
            Метрика(и) оценки качества модели
        model : BaseEstimator
            Базовая модель для оптимизации

    Returns:
        Dict[str, np.ndarray]
            Результаты вложенной кросс-валидации
    """
    
    X = df_rrs.loc[:, name_bands]
    target = df_rrs['CHL'].to_numpy().ravel()
    grid_model = GridSearchCV(make_pipeline(StandardScaler(), model), param_grid, cv=5,
                            scoring=scoring, return_train_score=True).fit(X, target)
    
    results = do_cross_validation(grid_model, X, target)
    
    return results  

# Отбор признаков

In [42]:
rf = RandomForestRegressor(random_state=42)
rf.fit(df_rrs_suppl.iloc[:, :-1], df_rrs_suppl['CHL'])
importance = pd.Series(rf.feature_importances_, index=df_rrs_suppl.iloc[:, :-1].columns).sort_values(ascending=False)
top_features = importance.head(3).index.tolist()

In [43]:
C_range = np.logspace(-1, 7, 10, base=2)
gamma_range = np.logspace(-7, -3, 10, base=2)
param_grid = dict(svr__gamma=gamma_range, svr__C=C_range, svr__epsilon=[0.1, 0.01])

param_grid_rf = {
    'randomforestregressor__n_estimators': [32, 64, 128],
    'randomforestregressor__max_depth': [5, 10, 15],
    'randomforestregressor__max_features': ['sqrt', 0.3],
    'randomforestregressor__min_samples_split': [2, 5]
}

# Сравнение моделей

In [46]:
for i in range(1, len(importance) + 1):
    current_features = importance.head(i).index.tolist()
    print(f"{current_features}:")
    nested_cv(df_rrs_suppl, current_features, param_grid, 'neg_root_mean_squared_error', SVR())
    print()

['BGI']:
Cross-validated Bias: (0.30 + 0.34 + 0.38 + -0.17 + -0.40) / 5 = 0.09
Cross-validated RMSE: (-0.69 + -0.74 + -0.74 + -0.57 + -0.51) / 5 = -0.65
Cross-validated MAPE: (-0.51 + -0.49 + -0.34 + -0.81 + -0.53) / 5 = -0.53

['BGI', 'NIRRI']:
Cross-validated Bias: (0.24 + 0.19 + 0.24 + -0.16 + -0.41) / 5 = 0.02
Cross-validated RMSE: (-0.68 + -0.73 + -0.70 + -0.57 + -0.53) / 5 = -0.64
Cross-validated MAPE: (-0.61 + -0.56 + -0.41 + -0.92 + -0.55) / 5 = -0.61

['BGI', 'NIRRI', 'Ratio B3/B8']:
Cross-validated Bias: (0.26 + 0.32 + 0.29 + -0.16 + -0.40) / 5 = 0.06
Cross-validated RMSE: (-0.72 + -0.75 + -0.73 + -0.57 + -0.52) / 5 = -0.66
Cross-validated MAPE: (-0.64 + -0.51 + -0.40 + -0.87 + -0.55) / 5 = -0.59

['BGI', 'NIRRI', 'Ratio B3/B8', 'NDTI']:
Cross-validated Bias: (0.29 + 0.28 + 0.39 + -0.05 + -0.34) / 5 = 0.11
Cross-validated RMSE: (-0.69 + -0.75 + -0.75 + -0.55 + -0.45) / 5 = -0.64
Cross-validated MAPE: (-0.58 + -0.51 + -0.38 + -0.76 + -0.47) / 5 = -0.54

['BGI', 'NIRRI', 'Ratio

In [47]:
for i in range(1, len(importance) + 1):
    current_features = importance.head(i).index.tolist()
    print(f"{current_features}:")
    nested_cv(df_rrs_suppl, current_features, param_grid_rf, 'neg_root_mean_squared_error', RandomForestRegressor())
    print()

['BGI']:
Cross-validated Bias: (0.19 + 0.15 + 0.27 + -0.16 + -0.35) / 5 = 0.02
Cross-validated RMSE: (-0.59 + -0.68 + -0.68 + -0.57 + -0.51) / 5 = -0.61
Cross-validated MAPE: (-0.36 + -0.49 + -0.38 + -0.90 + -0.45) / 5 = -0.52

['BGI', 'NIRRI']:
Cross-validated Bias: (0.14 + 0.14 + 0.23 + -0.21 + -0.34) / 5 = -0.01
Cross-validated RMSE: (-0.56 + -0.76 + -0.68 + -0.57 + -0.46) / 5 = -0.61
Cross-validated MAPE: (-0.46 + -0.57 + -0.36 + -0.95 + -0.43) / 5 = -0.56

['BGI', 'NIRRI', 'Ratio B3/B8']:
Cross-validated Bias: (0.17 + 0.24 + 0.30 + -0.22 + -0.37) / 5 = 0.03
Cross-validated RMSE: (-0.65 + -0.75 + -0.74 + -0.56 + -0.47) / 5 = -0.64
Cross-validated MAPE: (-0.53 + -0.52 + -0.39 + -0.95 + -0.47) / 5 = -0.57

['BGI', 'NIRRI', 'Ratio B3/B8', 'NDTI']:
Cross-validated Bias: (0.13 + 0.21 + 0.35 + -0.22 + -0.36) / 5 = 0.02
Cross-validated RMSE: (-0.63 + -0.77 + -0.75 + -0.59 + -0.46) / 5 = -0.64
Cross-validated MAPE: (-0.52 + -0.54 + -0.35 + -0.99 + -0.47) / 5 = -0.57

['BGI', 'NIRRI', 'Rati